In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import inshallah as ins
from sklearn.impute import KNNImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from geopy.distance import distance

ya allah


In [2]:
data = pd.read_csv("../CSV/big_files/ais_train.csv", sep="|")
vessels = pd.read_csv("../CSV/vessels.csv", sep="|")
test_og = pd.read_csv("../CSV/ais_test.csv")

vessels.head()

,shippingLineId,vesselId,CEU,DWT,GT,NT,vesselType,breadth,depth,draft,enginePower,freshWater,fuel,homePort,length,maxHeight,maxSpeed,maxWidth,rampCapacity,yearBuilt
0,61a8e672f9cba188601e84ab,61e9f38eb937134a3c4bfd8b,6500,21200.0,58684,17606.0,83.0,32.0,22.20,NaN,0.0,NaN,NaN,OSLO,199.00,5.0,18.6,15.2,150.0,2000
1,61ec94f1a8cafc0e93f0e92a,61e9f38eb937134a3c4bfd8d,4902,12325.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,MONROVIA,182.00,NaN,NaN,NaN,NaN,2006
2,61e213d5d612676a0f0fb755,61e9f38eb937134a3c4bfd8f,5000,13059.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,SAINT JOHN'S,182.00,NaN,NaN,NaN,NaN,2010
3,61be24574ea00ae59d0fe388,61e9f38eb937134a3c4bfd91,4200,12588.0,39362,NaN,83.0,28.0,NaN,NaN,11060.0,NaN,NaN,NaN,167.00,NaN,NaN,NaN,NaN,2011
4,61a8e673f9cba188601e84ae,61e9f390b937134a3c4bfd93,7450,21052.0,75528,24391.0,83.0,37.2,22.23,NaN,13140.0,491.47,3236.78,Panama,199.98,NaN,NaN,NaN,NaN,2018


In [3]:
def createIntervals(data0, FEATURES, n=5):
    data = data0.copy()
    data.sort_values("time", inplace=True)
    data["day_time"] = data["time"].apply(lambda t: pd.Timestamp(str(t).split(" ")[0]))

    data["month"] = data["time"].apply(lambda t: t.month)
    data["day"] = data["time"].apply(lambda t: t.day)
    data["hour"] = data["time"].apply(lambda t: t.hour)

    data["time"] = (data["time"] - pd.Timestamp("2024-01-01 00:00:00")).dt.total_seconds()

    intervals, sizes, final = [], [], []
    
    for i, grp in data.groupby(pd.Grouper(key="day_time", freq=f"{n}D")):
        intervals.append(grp[FEATURES].values)
        sizes.append(len(grp[FEATURES].values))

    maxSize = max(sizes)

    for i in intervals:
        residual = maxSize - len(i)
        padded = np.pad(i, ((0, residual), (0,0)), mode="constant", constant_values=0)
        final.append(padded)

    return np.array(final), sizes

def normalize_lat_lon(lat, lon):
    """
    Normalizes extreme latitude and longitude values, ensuring:
    - Latitude is constrained between [-90, 90] with appropriate longitude shifts.
    - Longitude is wrapped to be within [-180, 180].
    """
    
    # Handle extreme latitude values
    while lat > 90 or lat < -90:
        if lat > 90:
            lat = 180 - lat
            lon += 180
        elif lat < -90:
            lat = -180 - lat
            lon += 180
    
    # Normalize longitude using modulo to bring it within [-180, 180]
    lon = ((lon + 180) % 360) - 180
    
    return lat, lon

def reconstruct(sol: pd.DataFrame) -> pd.DataFrame:
    sol["vesselId"] = last_int["vesselId"]
    sol["time"] = last_int["time"]

    not_last_int = cleaned[["time", "vesselId", "latitude", "longitude"]].head(cleaned.shape[0] - y_sizes[-1])
    sol["latitude"] = np.nan

    mongo = []

    for i in sol["vesselId"].unique():
        current = sol[sol["vesselId"] == i]
        last_lat = not_last_int[not_last_int["vesselId"] == i]["latitude"].iloc[-1]
        last_lon = not_last_int[not_last_int["vesselId"] == i]["longitude"].iloc[-1]

        current["latitude"] = current["delta_lat"].cumsum() + last_lat
        current["longitude"] = current["delta_lon"].cumsum() + last_lon

        for _,row in current.iterrows():
            mongo.append(row.to_dict())

    truth = pd.DataFrame(mongo)
    return truth


In [15]:
#Imputation
vesselData = vessels.copy()
imputer = KNNImputer(n_neighbors=15)

non_numeric = ["vesselId", "shippingLineId", "homePort"]
non = vesselData[non_numeric]
vesselData.drop(columns=non_numeric, inplace=True)

cols, ind = vesselData.columns, vesselData.index

vesselData = pd.DataFrame(imputer.fit_transform(vesselData), columns=cols, index=ind)

for i in non_numeric:
    vesselData[i] = non[i]

#Cleaning   
speeds = {}
for _,row in vesselData.iterrows():
    speeds[row["vesselId"]] = row["maxSpeed"]

merged = []
data["maxSpeed"] = np.nan

for i in data["vesselId"].unique():
    boat = data[data["vesselId"] == i]
    boat["maxSpeed"] = speeds[i]

    for _,row in boat.iterrows():
        merged.append(row.to_dict())

merged = pd.DataFrame(merged, columns=data.columns)
merged.sort_values("time", inplace=True)


In [18]:
cleaned = ins.cleanUp(merged, n=5)

# cleaned["volume"] = cleaned["length"]*cleaned["breadth"]*cleaned["depth"]

cleaned.to_csv("cleaned.csv", sep="|")
cleaned.head()

print(data["vesselId"].unique()[0:6])
print(cleaned["vesselId"].unique()[0:6])

['61e9f3a8b937134a3c4bfdf7' '61e9f3d4b937134a3c4bff1f'
 '61e9f436b937134a3c4c0131' '61e9f3b4b937134a3c4bfe77'
 '61e9f41bb937134a3c4c0087' '61e9f468b937134a3c4c028f']
['61e9f436b937134a3c4c0131' '61e9f3d4b937134a3c4bff1f'
 '61e9f3b4b937134a3c4bfe77' '61e9f3a8b937134a3c4bfdf7'
 '61e9f41bb937134a3c4c0087']


In [ ]:
#Normalize
NORMAL = ["delta_time", "speed_from_prev", "dist_from_prev", "delta_lat_lag_1", "delta_lon_lag_1",
          "delta_lat_cum", "delta_lon_cum", "delta_lat", "delta_lon"]

mean, std = cleaned[NORMAL].mean(), cleaned[NORMAL].std()
cleaned[NORMAL] = (cleaned[NORMAL] - mean) / std

cleaned[NORMAL].head()

In [ ]:
#Gen data
TARGETS = ["delta_lat", "delta_lon", "delta_lat_cum", "delta_lon_cum"]
FEATURES = [x for x in NORMAL if x not in TARGETS]

X, X_sizes = createIntervals(cleaned, FEATURES)
y, y_sizes = createIntervals(cleaned, TARGETS)

X_train, X_test = X[:-1], X[-1].reshape((1, X.shape[1], X.shape[2]))
y_train, y_test = y[:-1], y[-1]

#Parameters
sequence_length = X.shape[1]
num_features = X.shape[2]
num_targets = y.shape[2]
print(X.shape)

In [ ]:
#LSTM
def lstm():
    lstm = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, input_shape=(sequence_length, num_features), return_sequences=True),
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.Dense(num_targets)
    ])

    lstm.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    lstm.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_pred_lstm = lstm.predict(X_test)

    #Loss
    y_lstm = y_pred_lstm.reshape(y_test.shape)
    print(f"LSTM MSE: {mean_squared_error(y_test, y_lstm)}")

    return y_lstm

#RNN
def rnn():
    rnn = tf.keras.Sequential([
        tf.keras.layers.SimpleRNN(64, activation="relu", input_shape=(sequence_length, num_features), return_sequences=True),
        tf.keras.layers.SimpleRNN(32, return_sequences=True),
        tf.keras.layers.Dense(num_targets)
    ])

    rnn.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    rnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_rnn = rnn.predict(X_test).reshape(y_test.shape)
    print(f"RNN MSE: {mean_squared_error(y_test, y_rnn)}")

    return y_rnn

#CNN
def cnn():
    cnn = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', padding="same", input_shape=(sequence_length, num_features)),
        tf.keras.layers.Conv1D(filters=32, kernel_size=2, activation='relu', padding="same"),
        tf.keras.layers.Conv1D(filters=16, kernel_size=2, activation='relu', padding="same"),
        tf.keras.layers.Conv1D(filters=2, kernel_size=1) 
    ])

    cnn.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    cnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_cnn = cnn.predict(X_test).reshape(y_test.shape)
    print(f"CNN MSE: {mean_squared_error(y_test, y_cnn)}")

    return y_cnn

#LSTM with more layers
def lstm_plus():
    lstm_plus = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, input_shape=(sequence_length, num_features), return_sequences=True),
        tf.keras.layers.LSTM(64, input_shape=(sequence_length, num_features), return_sequences=True),
        tf.keras.layers.LSTM(32, input_shape=(sequence_length, num_features), return_sequences=True),
        tf.keras.layers.Dense(50),
        tf.keras.layers.Dense(num_targets)
    ])

    lstm_plus.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    lstm_plus.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_lstm_plus = lstm_plus.predict(X_test).reshape(y_test.shape)
    print(f"LSTM Plus MSE: {mean_squared_error(y_test, y_lstm_plus)}")

    return y_lstm_plus

#Bidirectional
def bi():
    bi = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True), input_shape=(sequence_length, num_features)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Dense(num_targets)
    ])

    bi.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    bi.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_bi = bi.predict(X_test).reshape(y_test.shape)
    print(f"Bidirectional MSE: {mean_squared_error(y_test, y_bi)}")

    return y_bi

#Hybrid
def hybrid():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=(sequence_length, num_features)),
        tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(34, return_sequences=True),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(num_targets)
    ])

    model.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_hybrid = model.predict(X_test).reshape(y_test.shape)
    print(f"Hybrid MSE: {mean_squared_error(y_test, y_hybrid)}")

    return y_hybrid, model


In [ ]:
# x = 0.7
# y_ensemble = x*y_cnn + (1-x)*y_hybrid
# print(f"Ensemble ({x}*CNN and {(1-x)}*LSTM) MSE: {mean_squared_error(y_test, y_ensemble)}")

y_hybrid, model = hybrid()
model.save("hybrid.keras")

In [ ]:
last_int = cleaned[["time", "vesselId", "latitude", "longitude" ,"delta_lat", "delta_lon"]].tail(y_sizes[-1]).reset_index(drop=True)
prepred = pd.DataFrame(y_hybrid[0:y_sizes[-1]], columns=["delta_lat", "delta_lon", "delta_lat_cum", "delta_lon_cum"])
pretruth = pd.DataFrame(y_test[0:y_sizes[-1]], columns=["delta_lat", "delta_lon", "delta_lat_cum", "delta_lon_cum"])

#Un normalize
prepred["delta_lat"] = (prepred["delta_lat"] * std[-1]) + mean[-1]
prepred["delta_lon"] = (prepred["delta_lon"] * std[-2]) + mean[-2]

pretruth["delta_lat"] = (pretruth["delta_lat"] * std[-1]) + mean[-1]
pretruth["delta_lon"] = (pretruth["delta_lon"] * std[-2]) + mean[-2]

#Reconstruction
truth = reconstruct(pretruth)
pred = reconstruct(prepred)

print(truth["vesselId"].unique())


In [ ]:
vessel = "61e9f41bb937134a3c4c0087"

fig = (
    ins.visualize_vessel_movements(truth[truth["vesselId"] == vessel])
)
fig.show()

fig = (
    ins.visualize_vessel_movements(cleaned[cleaned["vesselId"] == vessel])
)
fig.show()

fig = (
    ins.visualize_vessel_movements(pred[pred["vesselId"] == vessel])
)
fig.show()

In [ ]:
vessel = "61e9f41bb937134a3c4c0087"
cleaned = pd.read_csv("cleaned.csv", sep="|")

#Iterative style - Start Input: last row of train data.
#MSE lat/lon: 62
brur = cleaned.tail(y_sizes[-1]).copy()
brur = brur[brur["vesselId"] == vessel].reset_index(drop=True)
not_last_int = cleaned.head(cleaned.shape[0] - y_sizes[-1]).copy()

means, std = not_last_int[NORMAL].copy().mean(), not_last_int[NORMAL].copy().std()
not_last_int[NORMAL] = (not_last_int[NORMAL] - means) / std

#Delta Time
last_time = not_last_int[not_last_int["vesselId"] == vessel].iloc[-1]["time"]
first_delta_time = (brur.at[0, "time"] - last_time).total_seconds()

brur["time"] = (brur["time"] - pd.to_datetime("2024-01-01 00:00:00")).dt.total_seconds()
brur["delta_time"] = brur["time"] - brur["time"].shift(1)
brur.at[0, "delta_time"] = first_delta_time

delta_time = brur["delta_time"].to_numpy()

#Start predicting on last row
predictions, pos, inputs = [], [], []

next_input = not_last_int[not_last_int["vesselId"] == vessel].iloc[-1][FEATURES].to_numpy().reshape((1,num_features))
prev_pos = not_last_int[not_last_int["vesselId"] == vessel].iloc[-1][["latitude", "longitude"]].to_list()

rows = X_test.shape[1] - 1
padded = np.pad(next_input, pad_width=((0, rows), (0, 0)), mode='constant', constant_values=0).reshape(X_test.shape)
padded = np.asarray(padded).astype('float32')

next_pred = model.predict(padded)[0][0][0:2]
un_normalized = [(next_pred[0] * std[-2]) + means[-2], (next_pred[1] * std[-1]) + means[-1]]
predictions.append(np.array(un_normalized))

new_pos = np.add(prev_pos, un_normalized)
pos.append(new_pos)

#Speed, Dist and Lags
dist = distance((prev_pos[0], prev_pos[1]), (new_pos[0], new_pos[1])).km
speed = dist*1000 / brur.iloc[0]["delta_time"]
lat_lag, lon_lag = next_input[0][-2], next_input[0][-1]

next_input = np.array([brur.iloc[0]["delta_time"], (speed - means[1]) - std[1], (dist - means[2]) - std[2], lat_lag, lon_lag]).reshape((1, num_features))
inputs.append(next_input)

for i in range(delta_time.shape[0]-1):
    rows = X_test.shape[1] - len(inputs)
    current = np.array(inputs[0:i+1]).reshape((i+1, num_features))

    padded = np.pad(current, pad_width=((0, rows), (0, 0)), mode='constant', constant_values=0).reshape(X_test.shape)
    padded = np.asarray(padded).astype('float32')

    #Next prediction
    next_pred = model.predict(padded)[0][i+1][0:2]
    un_normalized = [(next_pred[0] * std[-2]) + means[-2], (next_pred[1] * std[-1]) + means[-1]]
    predictions.append(np.array(next_pred))

    prev_pos = pos[i]
    new_pos = np.add(prev_pos, un_normalized)
    pos.append(new_pos)

    #Dist, speed and lags
    dist = distance((prev_pos[0], prev_pos[1]), (new_pos[0], new_pos[1])).km
    speed = dist*1000 / brur.iloc[i]["delta_time"]
    lat_lag, lon_lag = predictions[-2][0], predictions[-2][1]


    next_input = np.array([(delta_time[i+1] - means[0]) / std[0], (speed - means[1]) - std[1], (dist - means[2]) - std[2], lat_lag, lon_lag]).reshape((1, num_features))
    inputs.append(next_input)




In [ ]:
pred_arr = np.array(predictions)
df = pd.DataFrame(pred_arr, columns=["delta_lat", "delta_lon"])

df["vesselId"] = vessel
df["time"] = cleaned.tail(y_sizes[-1])[cleaned.tail(y_sizes[-1])["vesselId"] == vessel].reset_index(drop=True)["time"]

last_pos = not_last_int[not_last_int["vesselId"] == vessel].iloc[-1][["latitude", "longitude"]].to_list()
df["latitude"] = last_pos[0] + df["delta_lat"].cumsum()
df["longitude"] = last_pos[1] + df["delta_lon"].cumsum()

y_og = cleaned.tail(y_sizes[-1])[cleaned.tail(y_sizes[-1])["vesselId"] == vessel][["latitude", "longitude"]].copy().reset_index(drop=True)
y_og.head()
y_pred = df[["latitude", "longitude"]].copy().reset_index(drop=True)
print(f"MSE: {mean_squared_error(y_og, y_pred)}")
df[["vesselId", "time", "latitude", "longitude"]].to_csv("bigga_nalls.csv", sep="|")

In [ ]:
fig = (
    ins.visualize_vessel_movements(cleaned[cleaned["vesselId"] == vessel])
)
fig.show()

fig = (
    ins.visualize_vessel_movements(df[df["vesselId"] == vessel])
)
fig.show()

In [ ]:
vessel = "61e9f41bb937134a3c4c0087"

#Input: last block of train data
#MSE 8.8
brur = cleaned.tail(y_sizes[-1]).copy()
brur = brur[brur["vesselId"] == vessel].reset_index(drop=True)
not_last_int = cleaned.head(cleaned.shape[0] - y_sizes[-1]).copy()

means, std = cleaned[NORMAL].copy().mean(), cleaned[NORMAL].copy().std()
not_last_int[NORMAL] = (not_last_int[NORMAL] - means) / std

#Delta Time
last_time = not_last_int[not_last_int["vesselId"] == vessel].iloc[-1]["time"]
first_delta_time = (brur.at[0, "time"] - last_time).total_seconds()

brur["time"] = (brur["time"] - pd.to_datetime("2024-01-01 00:00:00")).dt.total_seconds()
brur["delta_time"] = brur["time"] - brur["time"].shift(1)
brur.at[0, "delta_time"] = first_delta_time

delta_time = brur["delta_time"].to_numpy()

brur[["vesselId", "delta_time"]].head()

# --- Predicting ---
# Setup
inputs, predictions, pos = [], [], []
no_rows = X_train[0].shape[0]

X_vessel, _ = createIntervals(cleaned, FEATURES + ["vesselId"])
X_vessel_last_df = pd.DataFrame(X_vessel[-2], columns=(FEATURES + ["vesselId"]))
vessel_df = X_vessel_last_df[X_vessel_last_df["vesselId"] == vessel].copy()
vessel_df[FEATURES] = (vessel_df[FEATURES] - means[0:5]) / std[0:5]
vessel_arr = vessel_df[FEATURES].to_numpy()


#Prediction time
i = 0
while i <= 0:
    #Add new predictions to the window
    if len(inputs) != 0:
        current = np.array(inputs).reshape((len(inputs), num_features))
        next_input = np.concatenate((vessel_arr, current), axis=0)
    else:
        next_input = vessel_arr

    #Pad
    remaining = no_rows - next_input.shape[0]
    next_input = [np.pad(next_input, pad_width=((0, remaining), (0,0)), mode="constant", constant_values=0)]
    next_input = np.asarray(next_input).astype("float32")

    #Predict
    next_pred = model.predict(next_input)

    inputs.append([i for j in range(num_features)])
    i += 1

#Hold on
next_pred = next_pred.reshape((next_pred.shape[1], next_pred.shape[2]))
next_pred = next_pred[0:delta_time.shape[0]]

df = pd.DataFrame(next_pred, columns=["delta_lat", "delta_lon", "shit1", "shit2"])
df.drop(columns=["shit1", "shit2"], inplace=True)
df["delta_lat"] = (df["delta_lat"] * std[-2]) + means[-2]
df["delta_lon"] = (df["delta_lon"] * std[-1]) + means[-1]

df["vesselId"] = vessel
df["time"] = cleaned.tail(y_sizes[-1])[cleaned.tail(y_sizes[-1])["vesselId"] == vessel].reset_index(drop=True)["time"]

last_pos = not_last_int[not_last_int["vesselId"] == vessel].iloc[-1][["latitude", "longitude"]].to_list()
df["latitude"] = last_pos[0] + df["delta_lat"].cumsum()
df["longitude"] = last_pos[1] + df["delta_lon"].cumsum()

y_og = cleaned.tail(y_sizes[-1])[cleaned.tail(y_sizes[-1])["vesselId"] == vessel][["latitude", "longitude"]].copy().reset_index(drop=True)
y_og.head()
y_pred = df[["latitude", "longitude"]].copy().reset_index(drop=True)
print(f"MSE: {mean_squared_error(y_og, y_pred)}")
df.head()

In [ ]:
fig = (
    ins.visualize_vessel_movements(cleaned[cleaned["vesselId"] == vessel])
)
fig.show()

fig = (
    ins.visualize_vessel_movements(df[df["vesselId"] == vessel])
)
fig.show()

In [ ]:
vessel = "61e9f41bb937134a3c4c0087"

#Iterative - Start Input: last block of train data
#
brur = cleaned.tail(y_sizes[-1]).copy()
brur = brur[brur["vesselId"] == vessel].reset_index(drop=True)
not_last_int = cleaned.head(cleaned.shape[0] - y_sizes[-1]).copy()

means, std = cleaned[NORMAL].copy().mean(), cleaned[NORMAL].copy().std()
not_last_int[NORMAL] = (not_last_int[NORMAL] - means) / std

#Delta Time
last_time = not_last_int[not_last_int["vesselId"] == vessel].iloc[-1]["time"]
first_delta_time = (brur.at[0, "time"] - last_time).total_seconds()

brur["time"] = (brur["time"] - pd.to_datetime("2024-01-01 00:00:00")).dt.total_seconds()
brur["delta_time"] = brur["time"] - brur["time"].shift(1)
brur.at[0, "delta_time"] = first_delta_time

delta_time = brur["delta_time"].to_numpy()

brur[["vesselId", "delta_time"]].head()

# --- Predicting ---
# Setup
inputs, predictions, pos = [], [], []
no_rows = X_train[0].shape[0]

X_vessel, _ = createIntervals(cleaned, FEATURES + ["vesselId"])
X_vessel_last_df = pd.DataFrame(X_vessel[-2], columns=(FEATURES + ["vesselId"]))

vessel_df = X_vessel_last_df[X_vessel_last_df["vesselId"] == vessel].copy()
vessel_df[FEATURES] = (vessel_df[FEATURES] - means[0:5]) / std[0:5]
vessel_arr = vessel_df[FEATURES].to_numpy()

init_pos = not_last_int[not_last_int["vesselId"] == vessel].iloc[-1][["latitude", "longitude"]].to_list()
init_lags = not_last_int[not_last_int["vesselId"] == vessel].iloc[-1][["delta_lat", "delta_lon"]].to_numpy()

pos.append(init_pos)
predictions.append(init_lags)

#Prediction time
i = 0
while i < delta_time.shape[0]:
    #Add new predictions to the window
    if len(inputs) != 0:
        current = np.array(inputs).reshape((len(inputs), num_features))
        next_input = np.concatenate((vessel_arr, current), axis=0)
    else:
        next_input = vessel_arr

    #Pad
    remaining = no_rows - next_input.shape[0]
    next_input = [np.pad(next_input, pad_width=((0, remaining), (0,0)), mode="constant", constant_values=0)]
    next_input = np.asarray(next_input).astype("float32")

    #Predict
    next_pred = model.predict(next_input)
    relevant = next_pred[0][(vessel_arr.shape[0]) + i][0:2]
    predictions.append(np.array(relevant))

    #Calculations
    un_normalized = [(relevant[0] * std[-2]) + means[-2], (relevant[1] * std[-1]) + means[-1]]

    prev_pos = pos[i]
    new_pos = np.add(prev_pos, un_normalized)
    pos.append(new_pos)

    #Dist, speed and lags
    dist = distance((prev_pos[0], prev_pos[1]), (new_pos[0], new_pos[1])).km
    speed = dist*1000 / delta_time[i]
    lat_lag, lon_lag = predictions[-2][0], predictions[-2][1]

    next_input_row = np.array([(delta_time[i] - means[0]) / std[0], (speed - means[1]) - std[1], (dist - means[2]) - std[2], lat_lag, lon_lag]).reshape((1, num_features))

    inputs.append(next_input_row)
    i += 1

pos = np.array(pos[1::])
print(pos.shape, delta_time.shape)


In [ ]:
df = pd.DataFrame(pos, columns=["latitude", "longitude"])

df["vesselId"] = vessel
df["time"] = cleaned.tail(y_sizes[-1])[cleaned.tail(y_sizes[-1])["vesselId"] == vessel].reset_index(drop=True)["time"]

y_og = cleaned.tail(y_sizes[-1])[cleaned.tail(y_sizes[-1])["vesselId"] == vessel][["latitude", "longitude"]].copy().reset_index(drop=True)
y_og.head()
y_pred = df[["latitude", "longitude"]].copy().reset_index(drop=True)
print(f"MSE: {mean_squared_error(y_og, y_pred)}")
df.head()

In [ ]:
fig = (
    ins.visualize_vessel_movements(cleaned[cleaned["vesselId"] == vessel])
)
fig.show()

fig = (
    ins.visualize_vessel_movements(df[df["vesselId"] == vessel])
)
fig.show()

In [ ]:
biggie = pd.read_csv("bigga_nalls.csv", sep="|")
biggie.head()

In [ ]:
test = test_og.copy()
test["time"] = pd.to_datetime(test["time"])
test.drop(columns=["scaling_factor"], inplace=True)

cleaned = pd.read_csv("cleaned.csv", sep="|")
cleaned["time"] = pd.to_datetime(cleaned["time"])


In [ ]:
for i in test["vesselId"].unique()[0:5]:
    boat = test[test["vesselId"] == i].reset_index(drop=True)
    boat_train = cleaned[cleaned["vesselId"] == i].reset_index(drop=True)

    # last_time = boat_train
    # boat["delta_time"] = (boat["time"] - boat["time"].shift(1))
    # boat.at[0, "delta_time"] = boat.at[0, "time"] - last_time
    # boat["delta_time"] = boat["delta_time"].dt.total_seconds()

    print(boat_train.head(2))


In [ ]:
print(len(data["vesselId"].unique()))